In [33]:
import os
from pandas import read_csv, read_excel, DataFrame
import sae
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum as qsum
import numpy as np

import sys
sys.path.append('../')
import saedfsc

Here is a notional list of suppliers.

In [34]:
saedfsc.suppliers

,Name,Location,Rating,SetUpCost
0,Supplier1,Location1,1.178742,4303
1,Supplier2,Location2,2.930649,2174
2,Supplier3,Location1,4.821298,2699
3,Supplier4,Location4,2.837146,2724
4,Supplier5,Location2,3.659921,4715
5,Supplier6,Location2,1.560158,2056
6,Supplier7,Location3,1.165378,3938


Customer information

In [35]:
saedfsc.customers

,Name,Quantity,PriceFocus,PerformanceUtilityWeights
0,CustomerType1,819,0.318619,"[0.28349404,0.15300543,0.21419679,0.00919576,0..."
1,CustomerType2,286,0.632910,"[0.29828083,0.40322026,0.08305298,0.0481549 ,0..."
2,CustomerType3,477,0.719322,"[0.40060376,0.9527889 ,0.52832218,0.363823 ,0..."
3,CustomerType4,722,0.006590,"[0.33949413,0.69433298,0.21546699,0.49446729,0..."
4,CustomerType5,323,0.191589,"[0.94294474,0.62223732,0.85902413,0.37645592,0..."
5,CustomerType6,589,0.805729,"[0.33270481,0.15287969,0.92995122,0.72758596,0..."
6,CustomerType7,837,0.583202,"[0.67515763,0.57397966,0.67347852,0.33828816,0..."
7,CustomerType8,193,0.352657,"[0.22915316,0.52918793,0.99310401,0.67754054,0..."
8,CustomerType9,90,0.160343,"[0.38814324,0.90425722,0.18675328,0.4983025 ,0..."
9,CustomerType10,526,0.338725,"[0.93348754,0.61676124,0.47003428,0.40255513,0..."


Quantity discount information

In [36]:
saedfsc.qtyDiscountSchedule

,Supplier,Part,PriceLevel,MinQty,Discount
0,Supplier1,1,1,5,0.20
1,Supplier1,1,2,10,0.30
2,Supplier1,1,3,15,0.40
3,Supplier1,2,1,20,0.25
4,Supplier1,2,2,30,0.35
5,Supplier1,2,3,40,0.45
6,Supplier2,1,1,25,0.25
7,Supplier2,1,2,35,0.35
8,Supplier2,1,3,45,0.45
9,Supplier2,3,1,25,0.30


In [37]:
partOptions = saedfsc.getPartOptionsWithSuppliers()

Chad: I put in numbers for placeholders. Please put the actual numbers in.

In [38]:
nominalPartPrices = {1 : 100, 2: 300, 3 : 400}

Parameters

In [39]:
productPrice = 20000
maxProductPrice = 25000
pricePerf = (maxProductPrice - productPrice) / maxProductPrice

Data structures

In [40]:
suppliers = saedfsc.suppliers['Name'].to_list()
supplierSetUpCost = dict(zip(saedfsc.suppliers['Name'], saedfsc.suppliers['SetUpCost']))
customers = saedfsc.customers['Name'].to_list()
cQty = dict(zip(saedfsc.customers['Name'], saedfsc.customers['Quantity'])) # customer quantities
cPriceFocus = dict(zip(saedfsc.customers['Name'], saedfsc.customers['PriceFocus']))
name_weights_dict = saedfsc.customers.set_index('Name')['PerformanceUtilityWeights'].to_dict()
cWts = {c : np.fromstring(name_weights_dict[c].strip('[]'), sep=',') for c in name_weights_dict}
unique_pairs_df = saedfsc.qtyDiscountSchedule[['Supplier', 'Part']].drop_duplicates()
suppliersPartPairs = list(unique_pairs_df.itertuples(index=False, name=None))

Chad: fill this in

In [41]:
objectives = ['mass', 'accel'] # fill this in with the objectives you want to optimize

Pricing schedule

In [42]:
priceLevels = {}
minQtys = {}
prices = {}
for s,p in suppliersPartPairs:
    filtered_df = saedfsc.qtyDiscountSchedule[(saedfsc.qtyDiscountSchedule['Supplier'] == s) & (saedfsc.qtyDiscountSchedule['Part'] == p)]
    priceLevels[s,p] = filtered_df['PriceLevel'].tolist()
    minQtys[s,p] = filtered_df.set_index('PriceLevel')['MinQty'].to_dict()

discounts = saedfsc.qtyDiscountSchedule.set_index(['Supplier','Part','PriceLevel'])['Discount'].to_dict()

maxQty = {}
for s,p in suppliersPartPairs:
    maxQty[s,p] = 1000

suppliersPartsAndLevels = []
for s,p in suppliersPartPairs:
    for l in priceLevels[s,p]:
        suppliersPartsAndLevels.append((s,p, l))

Create model container.

In [43]:
m = gp.Model()

Create variables

In [44]:
x = {} # part selection variables
for subsystem in partOptions.keys():
    x[subsystem] = m.addVars(partOptions[subsystem].index.values, vtype=GRB.BINARY, name="x[" + subsystem + "]")
y = m.addVars(suppliers, vtype=GRB.BINARY, name="y") # supplier selection variables
z = m.addVars(objectives, ub = 1, name="z") # objective variables (normalized to [0,1] for minimization)
v = m.addVars(suppliersPartsAndLevels, name="x") # the purchase quantity, given that the quantity of part p purchased from supplier s is in price level l
vBin = m.addVars(suppliersPartsAndLevels, vtype = GRB.BINARY, name="z") # if the quantity of part p purchased from supplier s is in price level l
w = m.addVars(customers, ub = cQty, name="w") # variables to hold customer demand

Set objective

In [45]:
m.setObjective(qsum(productPrice*cQty[c]*w[c] for c in customers) - qsum(nominalPartPrices[p]*discounts[s,p,l]*v[s,p,l] for s,p,l in suppliersPartsAndLevels) - y.prod(supplierSetUpCost), GRB.MAXIMIZE)

Quantity discount constraints

In [46]:
numLevels = {(s,p) : len(priceLevels[s,p]) for s,p in suppliersPartPairs}

suppliersPartsAndLevelsNotLast = [(s,p,l) for s,p in suppliersPartPairs for l in range(1,numLevels[s,p])]

m.addConstrs((minQtys[s,p][l]*vBin[s,p,l] <= v[s,p,l] for s,p,l in suppliersPartsAndLevels), 
                "PriceBreaks-LB")
m.addConstrs((v[s,p,l] <= minQtys[s,p][l+1]*vBin[s,p,l] for s,p,l in suppliersPartsAndLevelsNotLast), 
                "PriceBreaks-UB")
m.addConstrs((v[s,p,numLevels[s,p]] <= maxQty[s,p]*vBin[s,p,numLevels[s,p]] for s,p in suppliersPartPairs), 
                "PriceBreaks-UB-last");

Chad: add code here to set the equations for the $z$ variables (objectives)

Hugh: add code to compute customer demand based on utility

In [53]:
carDesignVec = [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
car = sae.COTSCar()
car.vector = carDesignVec
for c in customers:
    perf_utility = car.partworth_objectives(weights=cWts[c])[0]
    total_utility = (1-cPriceFocus[c])*perf_utility - cPriceFocus[c]*pricePerf
    print(c, total_utility)

CustomerType1 -0.7394093792153885
CustomerType2 -0.6533873162906297
CustomerType3 -0.8477211796557491
CustomerType4 -1.9210719585455573
CustomerType5 -1.665733246445088
CustomerType6 -0.4038530419231472
CustomerType7 -0.8608268241401282
CustomerType8 -1.1338668335802111
CustomerType9 -1.959438963581867
CustomerType10 -1.3769007126110848
CustomerType11 -0.40083411546904646
CustomerType12 -0.5324095760798941
CustomerType13 -1.1107635882588076
CustomerType14 -0.5049924677521397
CustomerType15 -0.7218970615056064
CustomerType16 -0.927649630968105
CustomerType17 -1.1571448864243878
CustomerType18 -0.8790330305616731
CustomerType19 -0.2697964815342995
CustomerType20 -1.414577745443255
CustomerType21 -1.066501413357933
CustomerType22 -0.9958148503748128
CustomerType23 -0.22654188293146127
CustomerType24 -0.37567515363943127
CustomerType25 -0.21617914089999674
